# FDA-hw3-2

## Target
Predict the game result from Dota 2 computer game.

Dataset: [Dota2 Games Results Data Set](https://archive.ics.uci.edu/ml/datasets/Dota2+Games+Results)



In [1]:
# Download the dataset
!wget -nc "https://archive.ics.uci.edu/ml/machine-learning-databases/00367/dota2Dataset.zip"
!unzip -n dota2Dataset.zip

--2020-06-01 14:49:56--  https://archive.ics.uci.edu/ml/machine-learning-databases/00367/dota2Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1821241 (1.7M) [application/x-httpd-php]
Saving to: ‘dota2Dataset.zip’

dota2Dataset.zip    100%[===================>]   1.74M  1.81MB/s    in 1.0s    

2020-06-01 14:49:58 (1.81 MB/s) - ‘dota2Dataset.zip’ saved [1821241/1821241]

Archive:  dota2Dataset.zip
  inflating: dota2Train.csv          
  inflating: dota2Test.csv           


## Attribute Information
1. Team won the game (-1 or 1)
2. Cluster ID (realated to location) **??**
3. Game Mode (e.g. All Pick)
4. Game Type (e.g. Ranked)
5. Indicator of hero (1 means team 1 use this hero, -1 means team -1 use this hero)

In [2]:
import pandas as pd
import numpy as np

def load_data(fileName):
  data = pd.read_csv(fileName, header=None, names=['Winning Team', 'Cluster', 'Mode', 'Type']+[f'hero {i}' for i in range(113)])
  data_y = data.loc[:, 'Winning Team']
  data_y = data_y.replace({-1:0})
  data_y = pd.get_dummies(data_y)
  data_x = data.loc[:, 'hero 0':]
  data_x = data_x.replace({0:None}) # Only one-hot encode 1 and -1 value, ignore 0 value
  data_x = pd.get_dummies(data_x, [f'hero {i}' for i in range(113)])
  return data_x, data_y

train_data_x, train_data_y = load_data('dota2Train.csv')
test_data_x, test_data_y = load_data('dota2Test.csv')

display(train_data_y)
display(train_data_x)

,0,1
0,1,0
1,0,1
2,0,1
3,0,1
4,1,0
...,...,...
92645,1,0
92646,0,1
92647,0,1
92648,1,0


,hero 0_-1,hero 0_1,hero 1_-1,hero 1_1,hero 2_-1,hero 2_1,hero 3_-1,hero 3_1,hero 4_-1,hero 4_1,hero 5_-1,hero 5_1,hero 6_-1,hero 6_1,hero 7_-1,hero 7_1,hero 8_-1,hero 8_1,hero 9_-1,hero 9_1,hero 10_-1,hero 10_1,hero 11_-1,hero 11_1,hero 12_-1,hero 12_1,hero 13_-1,hero 13_1,hero 14_-1,hero 14_1,hero 15_-1,hero 15_1,hero 16_-1,hero 16_1,hero 17_-1,hero 17_1,hero 18_-1,hero 18_1,hero 19_-1,hero 19_1,...,hero 92_-1,hero 92_1,hero 93_-1,hero 93_1,hero 94_-1,hero 94_1,hero 95_-1,hero 95_1,hero 96_-1,hero 96_1,hero 97_-1,hero 97_1,hero 98_-1,hero 98_1,hero 99_-1,hero 99_1,hero 100_-1,hero 100_1,hero 101_-1,hero 101_1,hero 102_-1,hero 102_1,hero 103_-1,hero 103_1,hero 104_-1,hero 104_1,hero 105_-1,hero 105_1,hero 106_-1,hero 106_1,hero 108_-1,hero 108_1,hero 109_-1,hero 109_1,hero 110_-1,hero 110_1,hero 111_-1,hero 111_1,hero 112_-1,hero 112_1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92645,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
92646,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
92647,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
92648,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


The occurences between train data and test data looks similar. So I think it's okay to preprocess the data without caring about different data distribution.

In [3]:
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch
NUM_EPOCHS=100


class Dota2Dataset(Dataset):
  def __init__(self, x, y):
    self.x = x
    self.y = y
  def __getitem__(self, idx):
    return self.x.iloc[idx].to_numpy(), self.y.iloc[idx].to_numpy()

  def __len__(self):
    return len(self.x)

train_dataloader = DataLoader(dataset=Dota2Dataset(train_data_x, train_data_y),
                         batch_size=128,
                         shuffle=True,
                         num_workers=4)
test_dataloader = DataLoader(dataset=Dota2Dataset(test_data_x, test_data_y),
                         batch_size=1024,
                         shuffle=True,
                         num_workers=4)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

class NN(nn.Module):
  def __init__(self, size):
    super(NN, self).__init__()
    self.size = size
    self.layer1 = nn.Sequential(
        nn.Linear(size, 10),
        nn.Sigmoid(),
        nn.Dropout(0.3)
    )
    self.bn1 = nn.BatchNorm1d(10)
    self.layer2 = nn.Sequential(
        nn.Linear(10, 2),
        nn.Sigmoid()
    )
  
  def forward(self, batch_x):
    x = self.layer1(batch_x)
    x = self.bn1(x)
    x = self.layer2(x)
    return x
    

neural_network = NN(train_data_x.shape[1])
optimizer = optim.Adam(neural_network.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.6)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(NUM_EPOCHS):
  n_correct, n_total = 0, 0
  neural_network.train()
  test_loss = []
  train_loss = []

  for idx, (train_x, train_y) in enumerate(train_dataloader):
    data, target = train_x.float(), train_y.float()
    optimizer.zero_grad()
    pred_y = neural_network(data).float()
    loss = criterion(pred_y, target)    
    train_loss.append(loss.item())
    n_correct += (torch.argmax(pred_y, dim=1) == torch.argmax(train_y, dim=1)).int().sum()
    n_total += len(train_x)

    loss.backward()
    optimizer.step()
  print(f'Train loss: {np.mean(train_loss)}')

  neural_network.eval()
  n_test_correct, n_test_total = 0, 0
  with torch.no_grad():
    for idx, (test_x, test_y) in enumerate(test_dataloader):
      pred_y = neural_network(test_x.float()).float()
      loss = criterion(pred_y, test_y.float())
      test_loss.append(loss.item())
      n_test_correct += (torch.argmax(pred_y, dim=1) == torch.argmax(test_y, dim=1)).int().sum()
      n_test_total += len(test_x)
  print(f'Test loss: {np.mean(test_loss)}')

  print(f"""
  Train acc: {float(n_correct)/n_total}, Test acc: {float(n_test_correct)/n_test_total}
  """)

  scheduler.step()

Train loss: 0.7052517453445255
Test loss: 0.6882919885895469

  Train acc: 0.536287101996762, Test acc: 0.5787837575286575
  
Train loss: 0.6868240737783317
Test loss: 0.6824276609854265

  Train acc: 0.5743227199136536, Test acc: 0.5966582475228288
  
Train loss: 0.6832822229486802
Test loss: 0.6777128577232361

  Train acc: 0.5833890987587695, Test acc: 0.5952010880124344
  
Train loss: 0.6820503058354499
Test loss: 0.678591560233723

  Train acc: 0.5851052347544522, Test acc: 0.5965611035554692
  
Train loss: 0.6815061091718094
Test loss: 0.6771628043868325

  Train acc: 0.5888289260658391, Test acc: 0.5964639595881096
  
Train loss: 0.68057488703596
Test loss: 0.6810345432975076

  Train acc: 0.5893901780895845, Test acc: 0.5954925199145134
  
Train loss: 0.6802880638047476
Test loss: 0.6791402318260886

  Train acc: 0.5923907177549919, Test acc: 0.5966582475228288
  
Train loss: 0.6803830289873629
Test loss: 0.6775071404197

  Train acc: 0.5944198596869941, Test acc: 0.59588109578